In [1]:
import os

In [2]:
%pwd

'c:\\Users\\knadim\\Desktop\\MLProjects\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\knadim\\Desktop\\MLProjects\\Kidney-Disease-Classification'

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [14]:
class Training:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        create_directories([self.config.training.root_dir])

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.training.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.params.IMAGE_SIZE[:-1],
            batch_size=self.params.BATCH_SIZE,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path(os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.params.AUGMENTATION:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=Path(os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")),
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.params.EPOCHS,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.training.trained_model_path,
            model=self.model
        )



In [15]:
try:
    training = Training()
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2025-09-16 18:36:34,249: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-16 18:36:34,251: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-16 18:36:34,251: INFO: common: created directory at: artifacts]
[2025-09-16 18:36:34,251: INFO: common: created directory at: artifacts/training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/150
23/23 [==============================] - 15s 613ms/step - loss: 10.3736 - accuracy: 0.5927 - val_loss: 5.5223 - val_accuracy: 0.6000
Epoch 2/150
23/23 [==============================] - 14s 606ms/step - loss: 5.2527 - accuracy: 0.7472 - val_loss: 0.7777 - val_accuracy: 0.9375
Epoch 3/150
23/23 [==============================] - 14s 610ms/step - loss: 4.2795 - accuracy: 0.7528 - val_loss: 1.6949 - val_accuracy: 0.5375
Epoch 4/150
23/23 [==============================] - 14s 607ms/step - loss: 2.9823 - accuracy: 0.7725 - val_loss: 0.7830 - val_accuracy: 0.8250
Epoch 5/150
